# Workshop Final - NPL - Clasificación de Sentimientos

**Integrantes del grupo:**

- Kevin Tasat
- Martha Alvarez
- Fernando Roa

### Resumen



El dataset a evaluar tiene un total de 16000 textos a los cuales se les asocian diferentes sentimientos o emociones. El objetivo de este trabajo es mediante análisis de lenguaje natural preparar la información y posteriormente entrenar un modelo de clasificación de emociones.

## Telegram BOT

### Cargue de librerías

In [1]:
import telegram.ext
import joblib
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns;sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.colors import ListedColormap
import statsmodels.api as sm
import math
from math import sqrt

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import svm
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report, auc
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

from IPython.display import Image
import itertools
import plotly.express as px
from itertools import chain
from pprint import pprint
from time import time
import logging

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from collections import defaultdict
#nltk.download('all')
#nltk.download('averaged_perceptron_tagger')

from collections import Counter
import re
import string

from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS

from textblob import TextBlob

from wordhoard import Antonyms
import random

from profanity_check import predict, predict_prob

pd.set_option('display.float_format', lambda x: '%.5f' % x)


C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearSVC from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your ow

In [3]:
# Se construye un diccionario con contracciones y sus respectivas palabras extendidas

contractions = { 
"ain't": "am not",
"aren't": "am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I had",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [4]:
df_stop_words = pd.read_csv('stopwords.csv')
stop_words = df_stop_words['stopwords'].tolist()

### Creación de clases

In [5]:
# Clase para identificar negación antes de una palabra
# transforma la negación en not y transforma la palabra en su antónimo
# Lo anterior trata de invertir el significado luego de pasar el texto por 
# stop words

class negation_transformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    @staticmethod
    # negate_sequence(text)
    #   text: sentence to process (creation of uni/bi
    #    is handled here)
    #
    # Detects negations and transforms negated words into 'not_' form
    #
    def negate_sequence(text):

        def antonyms_for(word):
            antonyms = set()
            for ss in wn.synsets(word):
                for lemma in ss.lemmas():
                    any_pos_antonyms = [ antonym.name() for antonym in lemma.antonyms() ]
                    for antonym in any_pos_antonyms:
                        antonym_synsets = wn.synsets(antonym)
                        if wn.ADJ not in [ ss.pos() for ss in antonym_synsets ]:
                            continue
                        antonyms.add(antonym)
            if antonyms==set():
                return word
            else:
                antonyms=list(antonyms)
                return antonyms[0]

        negation = False
        delims = "?.,!:;"
        result = []
        words = text.split()
        prev = None
        pprev = None
        for word in words:
            stripped = word.strip(delims).lower()
            negated = "not " + stripped if negation else stripped
            result.append(negated)

            if prev:
                bigram = prev + " " + negated
                pprev = prev
            prev = negated

            if any(neg in word for neg in ["not", "n't", "no"]):
                negation = not negation

            if any(c in word for c in delims):
                negation = False

        for elements in result:
            
            element=word_tokenize(elements)
            
            element0=0
            element_change=1
            
            if element[0]=="not" and len(element)>1 and element[1]!="feel":
                
                element0=" ".join(element)
                
                element_change=antonyms_for(element[1])
        
            result=[element_change if item == element0 else item for item in result]

        return " ".join(result)
    
    def fit(self, texto, y=None):
        return self

    def transform(self, texto):
        if isinstance(texto, pd.Series):
            texto=texto.apply(lambda x: self.negate_sequence(x))
        else:
            texto=pd.Series(texto)
            texto=texto.apply(lambda x: self.negate_sequence(x))
            
        return texto

In [6]:
# Clase para limpiar texto

class clean_texto(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    @staticmethod
    def clean_text(text):
        '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''

        # Convert words to lower case
        text = text.lower()

        # Replace contractions with their longer forms 
        if True:
            text = text.split()
            new_text = []
            for word in text:
                if word in contractions:
                    new_text.append(contractions[word])
                else:
                    new_text.append(word)
            text = " ".join(new_text)

        # Format words and remove unwanted characters
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text) 
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)

        # remove stop words
        #if remove_stopwords:
        text = text.split()
        stops = stop_words
        text = [w for w in text if not w in stops]
        text = " ".join(text)

        # spelling correction (tarda demasiado y no da buenos resultados)
        #text = TextBlob(text).correct() 
        #text = text.__str__() # opcion 1 (tarda demasiado)
        #text = '{}'.format(text) #opcion2 (tarda demasiado)
        #text = str(TextBlob(text).correct()) #opcion3 (tarda demasiado)

        #Lemmatize 

        #Defaultdict is a container like dictionaries present in the module collections. 
        #Defaultdict is a sub-class of the dictionary class that returns a dictionary-like object. 
        #The functionality of both dictionaries and defaultdict are almost same except for the fact that defaultdict never raises a KeyError. 
        #It provides a default value for the key that does not exists.
        
        tag_map = defaultdict(lambda : wn.NOUN)
        tag_map['J'] = wn.ADJ
        tag_map['V'] = wn.VERB
        tag_map['R'] = wn.ADV

        tokens = word_tokenize(text)
        lmtzr = WordNetLemmatizer()

        new_text2=[]

        for token, tag in pos_tag(tokens):
            lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
            new_text2.append(lemma)
            text=" ".join(new_text2)
        
        return text
    
    def fit(self, texto, y=None):
        return self

    def transform(self, texto):
        if isinstance(texto, pd.Series):
            texto=texto.apply(lambda x: self.clean_text(x))
        else:
            texto=pd.Series(texto)
            texto=texto.apply(lambda x: self.clean_text(x))
            
        return texto

In [7]:
# Cargamos el modelo

path_pipe = "pipe_model.sav"
pipe = joblib.load(path_pipe)

C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\PC\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Us

### Cargue del bot de telegram

In [8]:
with open('lala.py','r') as f:
    token=f.read()

In [9]:
updater=telegram.ext.Updater(token,use_context=True)
disp=updater.dispatcher

In [10]:
def start(update,context):
    update.message.reply_text("Hello I'm SentiBot, I'm here for you, please tell me how you feel")

In [11]:
def ayuda(update, context):
     update.message.reply_text("This is a Sentiment Analyzer Bot created by Group 2 of DH Academy")

In [12]:
def context(update, context):
     update.message.reply_text("This is a natural language processing machine learning built in Bot. It objective is to predict basic sentiments.")

In [13]:
def error(update,context):
    print(f"Update {update} caused error {context.error}")

In [14]:
def handle_message(update,context):  
    
    delims = "?.,!:;"

    user_message=update.message.text.strip(delims).lower()

    if user_message.isnumeric():

        return update.message.reply_text("That's not a feeling")
    
    elif user_message in ["hello","hi","whatsup","hola","hey","good afternoon"
                          ,"good morning", "good evening", "morning", "evening"
                          , "whatsup","hi there","hey there","howdy","sup"]:
        
        return update.message.reply_text("Hi! Nice to meet you. I'm here for you, please tell me how you feel")
    
    elif user_message in ["who are you","who are you?"]:
        return update.message.reply_text("I'm SentiBot")
    
    elif user_message in ["goodbye","bye","adios","chau"]:
        return update.message.reply_text("Goodbye, it was nice talking to you!")
    
    elif user_message in ["thanks","thank you","thank you very much"]:
        return update.message.reply_text("You are welcome. I hope I helped you")
    
    elif predict([str(user_message)])[0]==1:
        return update.message.reply_text("That's not very nice! let's try it again")

    else:

        prediction= pipe.predict(user_message)[0]

        if prediction == "anger":

            return update.message.reply_text(f'I detect {prediction} , remember "Anger makes you smaller, while forgiveness forces you to grow beyond what you were." - Chérie Carter-Scott.')

        elif prediction == "fear":

            return update.message.reply_text(f'I detect {prediction} , remember "Fear is the path to the dark side. Fear leads to anger. Anger leads to hate. Hate leads to suffering."- Yoda, The Phantom Menace.')

        elif prediction == "love":

            return update.message.reply_text(f'I detect {prediction} , remember "Love is a fruit in season at all times, and within reach of every hand." - Mother Teresa')

        elif prediction == "joy":

            return update.message.reply_text(f'I detect {prediction} , remember "A thing of beauty is a joy forever." ― John Keats, Endymion: A Poetic Romance')

        elif prediction == "surprise":

            return update.message.reply_text(f'I detect {prediction} , remember "Surprise is the greatest gift which life can grant us." - Boris Pasternak')

        elif prediction == "sadness":

            return update.message.reply_text(f'I detect {prediction} , remember "We must understand that sadness is an ocean, and sometimes we drown, while other days we are forced to swim." - R.M. Drake')

        else:

            return update.message.reply_text(f'I detect {prediction} but I don''t know')
    

In [15]:
disp.add_handler(telegram.ext.CommandHandler("start",start))
disp.add_handler(telegram.ext.CommandHandler("help",ayuda))
disp.add_handler(telegram.ext.CommandHandler("context",context))
disp.add_handler(telegram.ext.MessageHandler(telegram.ext.Filters.text,handle_message))
disp.add_error_handler(error)

In [16]:
updater.start_polling()
updater.idle()